# TP2 - Benchmark automatique

Dans ce TP nous allons définir une fonction pour mesurer les performances d'un modèle de langage via l'exécution de plusieurs benchmarks. Nous avons vu en cours trois manières de mesurer la performance d'un modèle de langage qu'on peut résumer à:
1. **Évaluation automatique**: via un ensemble de questions dont on connait la réponse
2. **Évaluation humaine**: qualification humaine de la réponse d'un modèle à une question
3. **Évaluation par modèle de langage**: notation ou comparaison de réponse d'un ou plusieurs modèles par un autre modèle

Nous nous intéressons ici au premier point, en particulier avec les benchmarks [GSM8K](https://huggingface.co/datasets/openai/gsm8k) et [HellaSwag](https://huggingface.co/datasets/Rowan/hellaswag).
Dans l'ensemble du notebook nous utiliserons la librairie LangChain.

Il est à garder en tête que ce notebook n'a qu'une portée pédagogique et n'est pas forcément à jour puisque le domaine évolue rapidement, ni que les pratiques sont celles validées par l'industrie.

## Uniformisation des benchmarks

Pour chaque benchmark que l'on considère, nous avons besoin de plusieurs informations :
* **Dataset** : une fonction pour charger les questions du benchmark
* **Référence** : une fonction capable d'identifier la réponse attentue
* **Prompt** : un prompt qui permet de demander correctement au modèle de répondre à la question
* **Chaîne** : une fonction qui renvoie la chaîne de traitement de LangChain
* **Score** : une fonction qui score la performance d'un modèle sur une question

Nous allons commencer par créer une classe qui regroupe ces desiderata :


In [ ]:
from langchain_core.runnables import Runnable
from langchain_core.prompts import PromptTemplate


class Benchmark:
    name: str

    def __init__(self, prompt: PromptTemplate):
        self.prompt = prompt

    def load_data(self):
        raise NotImplementedError

    def build_chain(self, model) -> Runnable:
        raise NotImplementedError

    def get_reference(self, sample):
        raise NotImplementedError

    def score(self, prediction, reference):
        raise NotImplementedError

Pour rendre cette classe plus concrète, commençons par travailler avec le benchmark [GSM8K](https://huggingface.co/datasets/openai/gsm8k).

### Benchmark GSM8K

On commence par charger le dataset et observer une question.

**Consigne** : Résoudre la question *à la main* et vérifier votre réponse. On recommande d'explorer plusieurs questions.

In [ ]:
import numpy as np; np.random.seed(42)
from datasets import load_dataset

dataset = load_dataset("gsm8k", "main")
dataset = dataset["test"]

print(f"Number of questions: {len(dataset)}")
index = 0
print("Example of question:\n", dataset[index]["question"])
print("And its answer:\n", dataset[index]["answer"])

Après avoir inspecté plusieurs éléments du dataset, on remarque que la réponse finale est placée après la chaîne de caractères "####".

**Consigne**: Construire une fonction `get_reference` qui prend en argument un élément de GMS8K (dictionnaire avec question et réponse) et renvoie la réponse attendue (string). On pourra utiliser la fonction [`search`](https://docs.python.org/3/library/re.html#re.search) de la librairie [`re`](https://docs.python.org/3/library/re.html#).
Puis tester cette fonction sur l'exemple précédent.

Il nous reste maintenant à définir un prompt tel que l'on puisse appeler un modèle et tester notre mécanique.

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["question"],
    template=(
        """You are a careful mathematician. Solve the problem step by step, then display your answer in the end.
        Question: {question}
        Answer:"""
    )
)

En intégrant l'appel à un modèle via Ollama sur notre ordinateur, on peut définir avec LangChain :

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import OllamaLLM

model = OllamaLLM(model="gemma3:4b")

chain = (
    {"question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

index = 0

question = dataset[index]["question"]
answer = get_reference(dataset[index])
response = chain.invoke(question)
print(f"Model answer : {response}")
print(f"The answer was : {answer}")


Il nous faut extraire la dernière valeur numérique pour obtenir automatiquement la réponse du modèle.

**Consigne** : Définir une fonction `score` qui prend en paramètre la réponse du modèle et la réponse attendue puis renvoie si les deux réponses sont identiques (1 / 0). On pourra utiliser la fonction [`findall`](https://docs.python.org/3/library/re.html#re.findall) de la librairie `re`.
Puis l'appliquer sur l'exemple précédent.

Nous avons l'ensemble des éléments nécessaire pour définir la classe `GSM8KBenchmark` depuis la classe `Benchmark` que nous avons défini précédemment.

**Consigne** : Définir cette classe comme sous-classe de `Benchmark`.

Il est maintenant temps de définir une fonction qui *fait* le benchmark.

**Consigne** : Définir une fonction `run_benchmark` qui prend en paramètre :
* `model_name` : le nom du modèle Ollama que l'on veut tester
* `benchmark` : la classe benchmark que l'on souhaite tester
* `max_samples` : le nombre maximum de questions que l'on souhaite utiliser

Puisque l'object avec lequel nous travaillons est un dataset HuggingFace, pour sélectionner $n$ lignes, on utilisera 
```python
dataset = dataset.select(range(max_samples))
```
De cette manière on préserve la structure.

**Consigne** : Utiliser la fonction `run_benchmark` en définissant un prompt pour GSM8K.

### HellaSwag

Maintenant que nous avons réussi à le faire pour le dataset GMS8K, attaquons-nous à [HellaSwag](https://huggingface.co/datasets/Rowan/hellaswag).

**Consigne** : En suivant la même approche que précédemment, implémenter une sous classe `HellaSwagBenchmark` à partir de la classe `Benchmark`. Puis utiliser la fonction `run_benchmark` pour valider votre travail.

## Réponses structurées

Sur quelques exemples tout semble fonctionner ! Mais il y a au moins une fragilité dans notre travail : la récupération de la réponse est peu fiable et largement dépendante des prompts.


Par exemple pour GMS8K, on aimerait avoir une réponse sous la forme d'un JSON :
```json
{
  "reasoning": "étapes de raisonnement",
  "final_answer": 18
}
```

De cette manière ce serait particulièrement simple d'extraire la réponse, sans pour autant ne pas avoir de *réflexion* du modèle. En revanche pour HellaSwag, un JSON extrêment simple suffit :
```json
{
    "choice": 2
}
```

Pour forcer le modèle à suivre ces formats, nous allons utiliser l'option [Pydantic](https://docs.langchain.com/oss/python/langchain/structured-output). Elle s'utilise comme suit, pour GSM8K :

In [ ]:
from pydantic import BaseModel, Field

class GSM8KOutput(BaseModel):
    reasoning: str = Field(description="Step-by-step reasoning")
    final_answer: float = Field(description="Final numeric answer")


Concernant l'intégration dans le prompt :

In [ ]:
from langchain.output_parsers import PydanticOutputParser

parser_gsm8k = PydanticOutputParser(pydantic_object=GSM8KOutput)

prompt_gsm8k = PromptTemplate(
    input_variables=["question"],
    partial_variables={"format_instructions": parser_gsm8k.get_format_instructions()},
    template=(
        """You are a careful mathematician. Solve the problem step by step.
        Question: {question}
        {format_instructions}"""
    ),
)

print(parser_gsm8k.get_format_instructions())

**Consigne** : Modifier la classe `Benchmark` et la sous-classe `GMS8KBenchmark` pour intégrer ces évolutions.

**Consigne** : Utiliser la fonction `run_benchmark` et vérifier que tout fonctionne.

**Consigne** : Réaliser la même modification pour HellaSwag, et vérifier que cela fonctionne.

## Pour aller plus loin

On pourrait implémenter d'autres benchmark, comparer vraiment des modèles entre eux, comparer des prompts entre eux...